## General idea
In the given paper variance reduction under without-replacement sampling methods are analysed. Popular without replacement sampling approaches are cyclic sampling, random reshuffling and shuffling once. This paper propose method called `Prox-DFinito` which is variant of [Finito](https://arxiv.org/pdf/1407.2710.pdf), which improves converges rates for without-replacement approaches.

## Problem statement
Strictly speaking our problem is Finite-sum composite optimization problem
$$
\min_{x\in R^d} F(x) + r(x)
$$
$$
F(x) = \frac{1}{n} \sum_{i=1}^{n} f_i(x)
$$

Where each $f_i(x)$ is convex and differentiable, $r(x)$ is regularisation function, which is convex but not necessary differentiable.

### Intuition

We can interpret $f_i(x)$ in Machine Learning as follows. $f$ is our loss function, weights of model is $x$ and $i$ is index of data row that we pass to our model. Our goal is to minimize average loss on our data with some regularisation in mind.

#### Useful Definitions/Notation


#### Sampling methods
Without replacement sampling garantues that each $f_i(x)$ will be presented only once during one epoch.
Without replacement methods which are considered in this paper:
- `Cyclic sampling`: $\pi = (\pi(1), \pi(2), \dots ,\pi(n))$ is random determined permutation of sample indexes. This order $\pi$ is same for all epochs
- `Random reshuffling`: every epoch new random permutation $\tau = (\tau(1), \tau(2), \dots, \tau(n))$ 

#### Notation
When we use $a^{b}$, $b$ means not power but iteration number

Let's define proximal operator
$$
prox_{\alpha r} (x) = argmin_{y \in R^d}(\alpha r(y) + \frac{1}{2} || y - x ||^2)
$$
Idea of operator is to choose such $y$ thay satisfy closeness to $x$ and regularisation term with respect to some hyperparameter $\alpha$

## Algorithm

$n$ - number of data rows

$d$ - number of weights in single model to optimize

Lets define $z_i$ as weights vector for single data row $i$. Thus, $z \in R^{n, d}$. Initially, can be random

Define $\bar z$ as: 
$$\bar z = \frac{1}{n} \sum_{i=1}^n z_i $$

Define $x \in R^{d}$ as our final weights for model

$\alpha$ is learning rate

$\theta$ is hyperparameter (see below for more)

![Alt text](./IAVFR_alg.png)

Let's breakdown algorithm by parts:

### Iteration

Idea of iteration is $x$ is computed as compromise ($prox_{\alpha r}$) between regularisation and some kind of average of best weights ($\bar z$) for every data row.

length of interval $[kn+1; (k+1)n]$ is $n$. It is written in this manner to store previous epoch in same iteration manner.

On every iteration $t$ we compute $x^{t-1}$ as proximal of $\bar z^{t-1}$ and regularisation function. 

Then we choose next data row index $i_t$, which can be not in numerical order. For example, if random shuffling is used.


Then we update:
$$
\begin{align*}
    z_i^t &=
    \begin{cases}
        x^{t-1} - \alpha \nabla f_i(x^{t-1}), & \text{ if } i = i_t, \\
        z_i^{t-1}, & \text{ if } i \neq i_t
    \end{cases} \\
    \bar z^t &= \bar z^{t-1} + \frac{z_{i_t}^t - z_{i_t}^{t-1}}{n}
\end{align*}
$$
(1) can be interpreted as updating only $z_{i_t}^t$

### Epoch end

After all iterations, we perform a 'damping' step.

$$
z_i^{(k+1)n} = (1 - \theta) z^{kn}_i + \theta z^{(k+1)n}_i\\
\bar z^{(k+1)n} = (1 - \theta) \bar z^{kn} + \theta \bar z^{(k+1)n}\\
$$

Here we update last weights matrix as weighted sum of first and last iteration weights. 